In [1]:
!pip3 install pandas dash

     |████████████████████████████████| 7.3 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 23.9 MB 32.2 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 77.4 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 79.2 MB/s eta 0:00:01
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3804 sha256=af0682cbc7ed2de0e69f365144d60e7b21ec9a2d072b82a8348f89898ceae98c
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/52/e4/f3/16724791571a955a46d54650510c98c04ab7d339626aee27cc
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4069 sha256=1343c6eb3264ccc3c9c5ad7592cbd3aaf76ade17c599b24fc10b3829fe060988
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/73/d8/8d/92f612c03c895f19bcc56a6c54be7bb41aaa698012a5624f60
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl s

In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2021-09-09 03:02:13--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2021-09-09 03:02:13 (104 MB/s) - ‘spacex_launch_dash.csv’ saved [3033/3033]



In [ ]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output
# from jupyter_dash import JupyterDash          # For execution in local installations of Jupyter Notebooks

spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

app = dash.Dash(__name__)
# app = JupyterDash(__name__)                 # For execution in local installations of Jupyter Notebooks

app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[
                                       {'label': 'All Sites',    'value': 'ALL'},
                                       {'label': 'CCAFS LC-40',  'value': 'CCAFS LC-40'},
                                       {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                       {'label': 'KSC LC-39A',   'value': 'KSC LC-39A'},
                                       {'label': 'VAFB SLC-4E',  'value': 'VAFB SLC-4E'}],
                                    value='ALL',
                                    placeholder="Select a Launch Site here",
                                    searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                html.P("Payload range (Kg):"),
                                
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    marks={
                                            0: '0',
                                         2500: '2500',
                                         5000: '5000',
                                         7500: '7500',
                                        10000: '10000'},
                                    value=[min_payload, max_payload]),
                                                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown',      component_property='value'))

def get_graph(launchSite):
    if launchSite == 'ALL':
        allSites = spacex_df[spacex_df['class']==1].groupby(['Launch Site'], as_index=False).sum()
        fig = px.pie(
            allSites, 
            values=allSites['class'], 
            names=allSites['Launch Site'], 
            title='Total Success Launches By Site')
        return fig
    else:
        singleSite = spacex_df[spacex_df['Launch Site']==launchSite]
        successRate = singleSite[['Launch Site', 'class']].value_counts()
        labels = [successRate.index[0][1], successRate.index[1][1]]
        fig = px.pie(
            successRate, 
            values=successRate, 
            names=labels, 
            title=f'Total Success Launches By Site {launchSite}')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
               [Input(component_id='site-dropdown', component_property='value'),
                Input(component_id='payload-slider', component_property='value')])

def get_graph(launchSite,slider):
    
    minPayload, maxPayload = slider
    
    plotData = spacex_df[spacex_df['Payload Mass (kg)']>=minPayload]
    plotData =   plotData[plotData['Payload Mass (kg)']<=maxPayload]
    
    if launchSite == 'ALL':
        fig = px.scatter(
            plotData, 
            x='Payload Mass (kg)', 
            y='class', 
            color='Booster Version Category',
            title='Correlation between Payload and Success for All Sites')
        return fig
    else:
        fig = px.scatter(
            plotData[plotData['Launch Site']==launchSite], 
            x='Payload Mass (kg)', 
            y='class', 
            color='Booster Version Category',
            title=f'Correlation between Payload and Success for Site {launchSite}')
        return fig
        
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)